original code found on https://pypi.org/project/drawSvg/

new code specific for our usecase

In [4]:
import drawSvg as draw
from drawSvg.widgets import DrawingWidget
from hyperbolic.poincare.shapes import *
from constructions import *
from Triangle import *
from deltaNeigbourhood import *

delta=0.3
click_list=[]
lineStyle2 = dict(stroke_width=0.01, stroke='white', fill='none' #,stroke_dasharray='.02,.02'
                 )

# Create drawing
d = draw.Drawing(2.2, 2.2, origin='center')
d.setRenderSize(500)
d.append(draw.Circle(0, 0, 1, fill='orange',stroke_width=0.01, stroke='black'))
g=draw.Group()
d.append(g)



                
def draw_thin_triangle(points):
    for i,x in enumerate(points):
        for y in points[i+1:]:
            p1=Point.fromEuclid(x[0],x[1])
            p2=Point.fromEuclid(y[0],y[1])
            line=Line.fromPoints(*p1,*p2,segment=True)
            g.draw(line, **lineStyle2)

def draw_thick_line(points):
    for i,x in enumerate(points):
        for y in points[i+1:]:
            p1=Point.fromEuclid(x[0],x[1])
            p2=Point.fromEuclid(y[0],y[1])
            line=Line.fromPoints(*p1,*p2,segment=True)
            g.draw(line, hwidth=0.4,fill='red')
    
def redraw_Mousepoint(points):
    for i,x in enumerate(points):
        p=Point.fromEuclid(x[0],x[1])
        g.draw(Circle.fromCenterRadius(p,0.02),fill='white')



def draw_delta_neigbourhood_twosides(points,delta):
    for i,x in enumerate(points):
        p1=Point.fromEuclid(x[0],x[1])
        g.draw(Circle.fromCenterRadius(p1, delta), fill='green',fill_opacity=1)
        for y in points[i+1:]:
            if i==0:
                p2=Point.fromEuclid(y[0],y[1])
                line = Line.fromPoints(*p1, *p2, segment=True)
                g.draw(line, hwidth=2*delta, fill='green',fill_opacity=1)
        


clicks=0
        
widget = DrawingWidget(d)
@widget.mousedown
def mousedown(widget, x, y, info):
        global clicks
        clicks+=1
        if (x**2 + y**2) ** 0.5 + 2e-10 < 1:
            if clicks<3:
                click_list.append((x, y))
            elif clicks==3:
                click_list.append((x,y))

                PList = []
                for y in click_list:
                    p=Point.fromEuclid(y[0],y[1])
                    PList.append(p)
                Tri=Triangle.fromVertices(PList)
                global delta
                delta = Tri.approx()

                g.children.clear()
                draw_delta_neigbourhood_twosides(click_list,delta)
                draw_thin_triangle(click_list)
            else:
                g.children.clear()
                draw_delta_neigbourhood_twosides(shift(click_list,clicks),delta)
                draw_thin_triangle(shift(click_list,clicks))
        widget.refresh()
        
        
@widget.mousemove
def mousemove(widget, x, y, info):
    if (x**2 + y**2) ** 0.5 + 2e-10 < 1:
        if len(click_list)<3:
            g.children.clear()
            draw_thin_triangle(click_list + [(x, y)])
            redraw_Mousepoint([(x,y)])
    widget.refresh()

widget


DrawingWidget()